Wordle Solver
=============

The idea is to take at each step the guess that would result in the lowest expected number of words that are still possible.

We do some precomputation with respect to that, by computing a matrix that returns an encoding of the result (position of all yellow and green letters) where in the row we have the correct word and in the column the guess.

In [1]:
import itertools

import numpy as np

In [2]:
# parse the word list
def file_to_list(f):
    return list({line.strip() for line in f})
with open('wordle_words.txt') as f:
    words = file_to_list(f)
len(words)

3448

In [3]:
# convert to a numpy array
def list_to_array(l):
    return np.array([[ord(c) for c in word] for word in l])
words_a = list_to_array(words)

In [4]:
# n x n x 5 array representing which letters are green when the guess (dim 9) is done against the true word (dim 1)

%time green = words_a[:, np.newaxis, :] == words_a[np.newaxis, :, :]  # numpy vectorization magic
green.shape

CPU times: user 166 ms, sys: 48.4 ms, total: 215 ms
Wall time: 177 ms


(3448, 3448, 5)

In [5]:
# quick check

print(words[0], words[2])
green[0, 2]

glass tires


array([False, False, False, False,  True])

In [6]:
# same, but for yellow. This requires some numpy black magic. But it's concise and fast.

%time yellow = (words_a[:, np.newaxis, :, np.newaxis] == words_a[np.newaxis, :, np.newaxis, :]).sum(axis=3).astype(bool)
yellow.shape

CPU times: user 1.84 s, sys: 173 ms, total: 2.01 s
Wall time: 2.01 s


(3448, 3448, 5)

In [7]:
print(words[4], words[6])
yellow[4,6]

grand crepe


array([False,  True, False, False, False])

In [8]:
def encode(a, base):
    """Encode the last dimension of an array as a single integer that is a function of its values.

    The way we do it is by encoding a 1d array x as sum(v * base**i for i, v in enumerate(x)).
    """

    powers = (base ** np.arange(a.shape[-1])).reshape(((1,) * (len(a.shape) - 1)) + (-1,))
    return (a * powers).sum(axis=-1)
encode(np.array([1, 4, 2]), 5)  # 1 + 4 * 5 + 2 * 25 = 71

71

In [9]:
# This is the encoded guess-result matrix. Consider each position as 3 (green), 1 (yellow) or 0 (grey)
# encoded[i, j] will be sum(v[k] * 4 ** k for k in range(5)) where v[k] is the value above for green/yellow/grey and k is the position

%time encoded = encode(green, 4) * 2 + encode(yellow, 4)
encoded.shape

CPU times: user 873 ms, sys: 212 ms, total: 1.09 s
Wall time: 1.08 s


(3448, 3448)

In [10]:
# here's what our matrix looks like. The visualization says we have 501 columns but it's a bug.

encoded

array([[1023,  880,  832, ...,    3,  372,    4],
       [ 816, 1023,  772, ...,    4,  304,    0],
       [ 768,  784, 1023, ...,   16,  256,   64],
       ...,
       [   3,   64,   64, ..., 1023,    0,    4],
       [ 369,   49,    1, ...,    0, 1023,  320],
       [   1,    0,    4, ...,   64,    1, 1023]])

In [11]:
def choose(encoded, active):
    """Choose a guess given that the columns in active are possible.

    Returns the index of the word to choose and the expected number of possible words after the guess.

    Could be improved by considering the possibility of choosing a word that's not possible.
    """

    best, best_score = 0, np.inf
    for i in active:
        row = encoded[i, active]
        bincount = np.bincount(row)  # number of possibilities for each result
        score = (bincount * (bincount - 1)).sum() / bincount.sum()  # expected number of still-valid alternatives
        if score < best_score:
            best, best_score = i, score
    return best, best_score

In [12]:
# The word with the best score as a first attempt

best, best_score = choose(encoded, np.arange(words_a.shape[0]))
print(words[best], best_score)

aloes 81.99245939675174


In [13]:
def play(word=None, verbose=True):
    """Play the game. If verbose, print stuff to see what's happening.

    Returns the number of guesses needed to get the right word.
    """

    if word is None:
        word = np.random.randint(len(words))
    # print(f"The word is {words[word].upper()}")
    active = np.arange(len(words))
    for i in itertools.count():
        guess, guess_score = choose(encoded, active)
        if verbose:
            print(f"Guess: {words[guess].upper()} (score {guess_score:.2f})")
            response = ['G' if g else 'Y' if y else '-'
                    for g, y in zip(green[guess, word], yellow[guess, word])]
            print(''.join(response))
        if guess == word:
            if verbose:
                print("Correct!")
            break
        active = active[encoded[guess, active] == encoded[guess, word]]  # update the possible words
        if verbose:
            candidates = ' '.join(words[candidate].upper() for candidate in active[:5])
            if active.size > 5:
                candidates = candidates + " ..."
            print(f"Remaining possible words: {active.size} ({candidates})")
        assert word in active
    return i + 1

In [14]:
# example play
play()

Guess: ALOES (score 81.99)
-Y-Y-
Remaining possible words: 50 (BELCH LEERY LIEGE LUNGE LITHE ...)
Guess: UTILE (score 2.08)
---YY
Remaining possible words: 6 (BELCH LEERY LEECH BERYL WELCH ...)
Guess: BELCH (score 0.33)
-GGGG
Remaining possible words: 1 (WELCH)
Guess: WELCH (score 0.00)
GGGGG
Correct!


4

In [15]:
# another one, 'cause we're getting curious and having some fun
play()

Guess: ALOES (score 81.99)
-G-Y-
Remaining possible words: 18 (FLUME FLUTE ELECT BLEND ELFIN ...)
Guess: ELITE (score 1.00)
YGG-G
Remaining possible words: 3 (GLIDE CLINE CLIME)
Guess: CLINE (score 0.00)
GGGGG
Correct!


3

In [16]:
# let's now do some brute forcing and find out average and max needed guesses

def bruteforce():
    """Try all possible words and return average, max needed guesses and worst case word."""
    max = s = 0
    worst_word = None
    for i in range(len(words)):
        if i % 100 == 0:
            print(f"word #{i}")
        guesses = play(i, False)
        s += guesses
        if guesses > max:
            max = guesses
            worst_word = i
    return s / len(words), max, worst_word

%time avg, max, worst_word = bruteforce()
print(f"Avg. guesses: {avg:.4f}; max attempts: {max} ({words[worst_word]})")

word #0
word #100
word #200
word #300
word #400
word #500
word #600
word #700
word #800
word #900
word #1000
word #1100
word #1200
word #1300
word #1400
word #1500
word #1600
word #1700
word #1800
word #1900
word #2000
word #2100
word #2200
word #2300
word #2400
word #2500
word #2600
word #2700
word #2800
word #2900
word #3000
word #3100
word #3200
word #3300
word #3400
CPU times: user 5min 19s, sys: 149 ms, total: 5min 19s
Wall time: 5min 20s
Avg. guesses: 3.8208; max attempts: 9 (caves)


In [17]:
play(worst_word)

Guess: ALOES (score 81.99)
Y--GG
Remaining possible words: 42 (GASES FAZES FAVES MATES BASES ...)
Guess: FATES (score 15.90)
-G-GG
Remaining possible words: 25 (GASES BASES WAVES CAPES BARES ...)
Guess: CAGES (score 7.60)
GG-GG
Remaining possible words: 6 (CAPES CASES CAKES CARES CANES ...)
Guess: CAPES (score 3.33)
GG-GG
Remaining possible words: 5 (CASES CAKES CARES CANES CAVES)
Guess: CASES (score 2.40)
GGYGG
Remaining possible words: 4 (CAKES CARES CANES CAVES)
Guess: CAKES (score 1.50)
GG-GG
Remaining possible words: 3 (CARES CANES CAVES)
Guess: CARES (score 0.67)
GG-GG
Remaining possible words: 2 (CANES CAVES)
Guess: CANES (score 0.00)
GG-GG
Remaining possible words: 1 (CAVES)
Guess: CAVES (score 0.00)
GGGGG
Correct!


9